<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Importing-Packages-and-Pulling-API-Data:" data-toc-modified-id="Importing-Packages-and-Pulling-API-Data:-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Importing Packages and Pulling API Data:</a></span></li><li><span><a href="#Parsing-JSON-Data:" data-toc-modified-id="Parsing-JSON-Data:-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Parsing JSON Data:</a></span></li><li><span><a href="#Data-Cleaning:" data-toc-modified-id="Data-Cleaning:-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Data Cleaning:</a></span></li><li><span><a href="#Mapping-Live-Data:" data-toc-modified-id="Mapping-Live-Data:-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Mapping Live Data:</a></span></li><li><span><a href="#Final-Map:" data-toc-modified-id="Final-Map:-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Final Map:</a></span></li></ul></div>

## Importing Packages and Pulling API Data:

In [31]:
import pandas as pd
import time
import requests
import folium
import leaflet
import warnings 
warnings.filterwarnings("ignore") 

In [2]:
ENV = pd.read_json("env.json", typ = "series")
API_KEY = ENV["API KEY"]

- Grabbing data from Tom Tom API via Traffic Incidents and Routing.

In [3]:
url = "https://api.tomtom.com/traffic/services/4/incidentDetails/s3/40.785091%2C-73.968285%2C40.665535%2C-73.969749/10/-1/json?projection=EPSG4326&expandCluster=true&originalPosition=true&key=" + API_KEY
url2 = "https://api.tomtom.com/routing/1/calculateRoute/40.785091%2C-73.968285%3A40.665535%2C-73.969749/json?computeBestOrder=true&routeRepresentation=polyline&computeTravelTimeFor=all&routeType=fastest&traffic=true&avoid=unpavedRoads&key=" + API_KEY
url


'https://api.tomtom.com/traffic/services/4/incidentDetails/s3/40.785091%2C-73.968285%2C40.665535%2C-73.969749/10/-1/json?projection=EPSG4326&expandCluster=true&originalPosition=true&key=zNPHiann1kO7gPHhTQLQGGZGDGlRDMrb'

In [4]:
res = requests.get(url)

In [5]:
res2 = requests.get(url2)

- Ensuring my requests are valid with 200 status.

In [6]:
res.status_code

200

In [7]:
res2.status_code

200

In [8]:
json_data_routing = res2.json()

In [9]:
json_data_traffic = res.json()

## Parsing JSON Data:

In [11]:
df_routing_1 = json_data_routing['routes'][0]['legs'][0]['points']

In [12]:
# tm The main Response element.
def parse_traffic_data(tm):
    #input: main response dictionary (tm)
    poi = parse_poi(tm['poi'])
    return poi

# poi (dict) - A single traffic incident, or a cluster of traffic incidents
def parse_poi(poi):
    """
    input: poi(dict) of traffic incidents
    return: dictionary {cluster_id: DataFrame of incident data}
    """
    data = {}
    for p in poi:
        data[p['id']] = parse_cpoi(p['cpoi'])
    return data

# cpoi - A single incident, only within an expanded cluster.
def parse_cpoi(cpoi):
#     """
#     input: cpoi(dict)
#     return: DataFrame of data
#     """
    return pd.DataFrame(cpoi) 

In [14]:
traffic = parse_traffic_data(json_data_traffic['tm'])

- Saving to Dataframes.

In [16]:
df_traffic = pd.concat([value for key,value in traffic.items()]) #grabs cluster id and cluster 

/Users/amybear/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [17]:
df_routing_1 = pd.DataFrame(df_routing_1) 

## Data Cleaning:

- Dropping unecessary columns.

In [18]:
df_traffic.drop(columns = ['op','dl','r','c'], inplace = True)

- Renaming columns to something more intelligible.

In [19]:
df_traffic= df_traffic.rename(columns={
'c': 'cause',
'cs': 'num_of_incidents',
'd':'description',
'ed': 'end_date',
'f': 'from',
't': 'to',
'ic': 'road_condition',
'l': 'length',
'p':'coord',
'ty': 'delay'
})

- Prepping columms to feed into map.

In [20]:
df_traffic['description'] = df_traffic['description'].str.capitalize()

- I created an interaction feature to to better capture the data I wanted to plot.

In [21]:
df_traffic['e'] = df_traffic['description'] +' '+ 'from' +' ' + df_traffic['from'] + ' ' + 'to'+ ' ' +  df_traffic['to']

In [22]:
incident_coor = df_traffic['coord']
incident_data = [df_traffic['e'], df_traffic['description']]

## Mapping Live Data:

- Creating a color assignment dictionary to feed into my for loop.

In [27]:
color_dict = {
   'Major event':'firebrick',
    'Bridge closed':'salmon',
     'Left lane closed':'dimgrey',
    'One lane closed':'slategrey',
    'Lane closed':'darkgoldenrod',
    'Right lane closed':'darkcyan',
   'Maintenance work':'coral',
    'Construction work':'darkorange',
   'Major roadworks':'crimson',
    'Stationary traffic':'sienna',
   'Roadworks':'slategrey',
   'Closed':'mediumslateblue',
    'Queuing traffic':'gold',
    'Slow traffic':'powderblue',
     'Lane blocked': 'goldenrod',
    'Broken down vehicle':'gold'
} 

- For loop that iterates through my traffic dataframe, cleans coords, assigns colors and maps my coordinates.

In [28]:
#Define coordinates of where we want to center our map
Manhattan_coords = [40.7831, -73.9712]

#Create the map
my_map2 = folium.Map(location = Manhattan_coords, zoom_start = 13)

for index, row in df_traffic.iterrows():
    name = row['e']
    incident_type = row['description']
    clean_coords = [row['coord']['y'], row['coord']['x']]#separating coords from dict and storing as [y,x]
    folium.CircleMarker(
        clean_coords,
        radius=8,
        popup = name,
        color='b',
        threshold_scale=[0,1,2,3],
        fill_color=color_dict[incident_type],
        fill=True,
        fill_opacity=0.8
        ).add_to(my_map2)

- Ploting my routing polyline.

In [29]:
folium.PolyLine(locations = df_routing_1, color="red", weight=2.5, opacity=1).add_to(my_map2)

## Final Map:

In [30]:
my_map2